# SQS / SNS FIFO (First In First Out)

## SQS FIFO
- 기본적으로 SQS는 순서를 보장하지 않음. (스탠다드 모드)
- 기본적으로 SQS는 메세지를 여러번 전달할 가능성 존재. (스탠다드 모드)
- FIFO 모드 : 메세지의 순서를 보장하며 단 한번만 전달
- 두가지 효과
    - 순서 보장, 한번만 전달
    - 중복 제거
    - 이외에 몇 기능 추가
- 단점 : 성능 저하
    - 예 : 300트랜젝션 per Second(FIFO) vs 거의 제한 없는 트랜젝션(Standard)
    - High Throughput 모드로 어느정도 완화 가능 (리전별로 최대 상한치가 다름)
    - 이름에 .fifo 필수 (예: my-demo-queue.fifo)

## SQS FIFO - Deduplication ID
- Deduplication ID : SQS에서 각 메세지의 중복 여부를 판단하기 위한 고유 토큰
- 메세지를 SQS에 전달할 때 부여할 수 있으며 한번 특정 Deduplication ID를 가진 메세지가 처리된 경우, 다음 5분동안 같은 Deduplication ID를 가진 메세지는 무시
    - 성공은 정상적으로 반환, 메세지만 무시
    - 메세지가 전달된 이후에도 계속 레코드를 추적
- 두가지 방법으로 제공
    - Contents Based : SQS에서 자동적으로 메세지의 Body의 SHA-256 해쉬를 Deduplication ID로 사용 ( = 바디 내용이 같으면 아이디가 같아짐)
        - 즉 Attribute는 해쉬 과정에 포함하지 않음. 바디가 같다면 어트리뷰트가 달라도 같은 메세지 취급
    - 메세지의 프로듀서가 직접 Deduplication ID를 생성해서 같이 전달(예: 타임스탬프 활용)

## Message Group ID (SQS, SNS 둘 다 해당)
- Message Group ID : SNS/SQS FIFO 내부에 있는 일종의 채널
- Message Group ID 단위로 순서 보장 및 전달이 이루어짐
    - 즉 다른 Message Group ID 끼리는 순서 보장이 이루어지지 않음
    - SQS FIFO 에서는 동일 Message Group ID를 가진 메세지는 동시에 하나만 처리 가능
        - 즉, 하나의 Message Group에서 맨 처음 메세지가 처리되지 않으면 나머지 Message Group 모두 대기
- SNS FIFO에서 Message Group ID를 전달했을 때, 대상이 SQS FIFO라면 Message Group ID 같이 전달

## SNS FIFO
- SNS의 메세지 전달을 FIFO로 할 수 있는 모드
- 두 가지 효과
    - 순서 보장
    - 중복 제거
- SQS FIFO / Standard Queue만 연동 가능
    - 즉 이메일, 휴대폰 SMS, HTTP 엔드포인트 등 일반적인 Subscriber와 연동 불가능
- 기타 기능
    - 메세지 그룹
    - 메세지 필터링
- 이름에 .fifo 필수(예: my-demo-topic.fifo)
- 메세지 순서
    - 일반적인 SNS + SQS 조합은 순서를 보장하지 않음
        - 즉 메세지가 발송된 순서와 별도로 대상이 수신
        - 일반 SNS / 일반 SQS 둘 다 순서 보장이 안됨
    - 순서를 보장하기 위해서는 SNS FIFO + SQS FIFO가 필요
    - 전체 Subscription에 전달되는 전체 메세지의 순서는 다를 수 있지만, 개별 Subscription이 받는 메세지들의 순서는 동일
    - Message Squence Number : 연속적은 아니지만 항상 증가하는 번호
        - SNS FIFO에서 메세지를 받아 발송할때 부여
        - Message Body에 포함 : 단 Raw Message 전달 활성화 시 포함하지 않음
- 필터링
    - 메세지 필터링을 통해 모든 메세지 대신 특정 메세지만 수신 가능
    - 각 Subscription Filter Policy 설정 가능
        - Filter Policy : 메세지의 바디 혹은 어트리뷰트 단위로 원하는 메세지를 매칭
        - 매칭 시 메세지 발송 / 불일치 시 메세지를 전달하지 않음

## SNS FIFO - Deduplication
- Deduplication : 중복 제거 : 정확하게 한번만 메세지를 전달하는 기능
- Deduplication ID를 기반으로 중복된 메세지를 제거
    - 5분 이내에 Deduplication ID가 같은 메세지가 전달되면 요청은 성공하지만 전달은 되지 않음
    - SNS 메세지가 전달된 이후에도 ID는 트래킹
- SNS에 전달된 메세지 바디를 기반으로 Deduplication ID 생성 가능 (바디를 해쉬, Attribute는 해싱에 반영되지 않음)

## SNS FIFO - Message Archive, Replay
- SNS FIFO에 전달된 메세지를 저장하고 필요에 따라 리플레이 가능
    - 사용 사례
        - 메세지 전달 과정의 오류를 복구
        - 기존 어플리케이션의 장애 복구
        - 신규 어플리케이션의 Sync 맞추기
- 메세지 보관 기간 설정 : 1일에서 365일
- 추가 보관 비용 발생 (저장 및 처리에 따라 과금)
- 리플레이 시 시간을 정해서 리플레이 가능